In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

#suppresses warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [13]:
tot_df = []
for j,i in enumerate(np.arange(2016, 2021, 1)):
    tot_df.append(pd.read_csv('../Data/Raw/CDC/CDCSTDcountysexrace'+ str(i) +'.csv', sep = ',', skiprows = 4))

In [17]:
df2016 = pd.DataFrame(tot_df[0])
df2017 = pd.DataFrame(tot_df[1])
df2018 = pd.DataFrame(tot_df[2])
df2019 = pd.DataFrame(tot_df[3])
df2020 = pd.DataFrame(tot_df[4])

,Indicator,Year,Geography,FIPS,Race/Ethnicity,Sex,Cases,Rate per 100000
0,Primary and Secondary Syphilis,2020 (COVID-19 Pandemic),"Abbeville County, SC",45001,White,Male,Data suppressed,Data suppressed
1,Primary and Secondary Syphilis,2020 (COVID-19 Pandemic),"Abbeville County, SC",45001,White,Female,Data suppressed,Data suppressed
2,Primary and Secondary Syphilis,2020 (COVID-19 Pandemic),"Abbeville County, SC",45001,Unknown,Male,Data suppressed,Data suppressed
3,Primary and Secondary Syphilis,2020 (COVID-19 Pandemic),"Abbeville County, SC",45001,Unknown,Female,Data suppressed,Data suppressed
4,Primary and Secondary Syphilis,2020 (COVID-19 Pandemic),"Abbeville County, SC",45001,Native Hawaiian/Other Pacific Islander,Male,Data suppressed,Data suppressed
